# MTrend Options

In [2]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [3]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_midcpnifty(f_today):

    days_to_thursday = (0 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [62]:
# bnf_pandas = pd.read_csv("../data/bnf_1hr_tv (9).csv")
bnf_pandas = pd.read_csv("../data/fin.csv")
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2023]
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')

In [63]:
bnf_pandas.tail()

,datetime,open,high,low,close,volume
869531,2024-06-07 15:25:00,22162.5000,22166.0500,22157.2500,22163.7000,0
869532,2024-06-07 15:26:00,22163.8000,22165.4000,22158.3500,22163.5000,0
869533,2024-06-07 15:27:00,22165.2500,22172.2000,22160.2000,22169.1500,0
869534,2024-06-07 15:28:00,22169.8500,22171.9500,22164.6500,22168.3000,0
869535,2024-06-07 15:29:00,22170.6000,22171.9000,22162.0000,22166.8000,0


In [64]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [65]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [66]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")

In [67]:
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [68]:
bnf.head()

open,high,low,close,volume,index,datetime
f64,f64,f64,f64,i64,datetime[ns],datetime[ns]
7442.4,7442.4,7398.05,7399.3,0,2017-01-02 09:15:00,2017-01-02 09:15:00
7399.7,7411.95,7399.7,7407.5,0,2017-01-02 09:16:00,2017-01-02 09:16:00
7407.7,7408.95,7385.75,7385.75,0,2017-01-02 09:17:00,2017-01-02 09:17:00
7386.05,7389.9,7382.4,7388.9,0,2017-01-02 09:18:00,2017-01-02 09:18:00
7389.45,7389.9,7382.05,7384.05,0,2017-01-02 09:19:00,2017-01-02 09:19:00


In [69]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(bnf, '5m', pd.Timedelta(minutes=0))

# bnf_1hr = ohlc_resampled
# bnf_final = bnf_1hr.to_pandas()
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final
# bnf_1hr

In [70]:
# def generate_signals1(df, signal_ma, trailing_ma):
#     df["c"] = pd.to_numeric(df["c"], errors="coerce")
#     df["MA20"] = df["c"].rolling(window=signal_ma).mean()
#     df["MA10"] = df["c"].rolling(window=trailing_ma).mean()

#     df["Buy_Signal"] = 0

#     # Generate signals using boolean masking
#     buy_signal_mask = ((
#         df["MA20"]
#         > df["MA20"].shift(1))
#         # & (df["MA20"].shift(1) > df["MA20"].shift(2))
#         # & (df['MA20'].shift(2) > df['MA20'].shift(3))
#         # & (df['MA20'].shift(3) > df['MA20'].shift(4))
#         # & (df['MA20'].shift(4) > df['MA20'].shift(5))
#     )

#     df.loc[buy_signal_mask, "Buy_Signal"] = 1

#     return df

In [71]:
def generate_signals1(df, signal_ma, trailing_ma, candles_in_num):
    df["c"] = pd.to_numeric(df["c"], errors="coerce")
    df["MA20"] = df["c"].rolling(window=signal_ma).mean()
    df["MA10"] = df["c"].rolling(window=trailing_ma).mean()

    df["Buy_Signal"] = 0

    # Generate signals using boolean masking
    buy_signal_mask = df["MA20"] > df["MA20"].shift(1)
    
    for i in range(1, candles_in_num):
        buy_signal_mask &= df["MA20"].shift(i) > df["MA20"].shift(i + 1)

    df.loc[buy_signal_mask, "Buy_Signal"] = 1

    return df

In [72]:
# def generate_signals2(df):
#     df["c"] = pd.to_numeric(df["c"], errors="coerce")
#     df["MA20"] = df["c"].rolling(window=20).mean()
#     df["MA10"] = df["c"].rolling(window=10).mean()

#     df["Sell_Signal"] = 0

#     # Generate signals using boolean masking
#     sell_signal_mask = (
#         df["MA20"]
#         < df["MA20"].shift(1)
#         & (df["MA20"].shift(1) < df["MA20"].shift(2))
#         # & (df["MA20"].shift(2) < df["MA20"].shift(3))
#         # & (df["MA20"].shift(3) < df["MA20"].shift(4))
#         # & (df["MA20"].shift(4) < df["MA20"].shift(5))
#         # & (df['MA20'].shift(5) < df['MA20'].shift(6))
#         # & (df['MA20'].shift(6) < df['MA20'].shift(7))
#     )

#     df.loc[sell_signal_mask, "Sell_Signal"] = 1

#     return df

In [73]:
# bnf1 = bnf
# bnf2 = bnf

# bnf1 = bnf1.to_pandas()
# bnf2 = bnf2.to_pandas()

# signals_df1 = generate_signals1(bnf1)
# signals_df2 = generate_signals2(bnf2)

# signals_df1 = signals_df1.rename(
#     columns={"open": "o", "high": "h", "low": "l", "close": "c"}
# )
# signals_df2 = signals_df2.rename(
#     columns={"open": "o", "high": "h", "low": "l", "close": "c"}
# )

In [74]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].tail())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

869531   2024-06-07 15:25:00
869532   2024-06-07 15:26:00
869533   2024-06-07 15:27:00
869534   2024-06-07 15:28:00
869535   2024-06-07 15:29:00
Name: datetime, dtype: datetime64[ns]


In [75]:
# GLOBAL VARIABLES

# INSTRUMENT = "BANKNIFTY"
# INDEX = "bnf"

# INSTRUMENT = "MIDCPNIFTY"
# INDEX = "midcpnifty"

INSTRUMENT = "FINNIFTY"
INDEX = "finnifty"

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"

PORTFOLIO_VALUE = 10_00_000
INDEX_LEV = 6

In [76]:
# data

In [77]:
async def trade_mtrend_option_buying(
    df, index, strike, asset_class, expiry, search_datetime, target
):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    eod_exit_flag = False
    remark = ""

    for i in range(0, len(df)):
        if df.iloc[i]['datetime'] >= search_datetime and df.iloc[i]['datetime'].time() <= dt.time(15, 20):
            # print(df.iloc[i]['datetime'])
            points = 0
            current_candle_open = df.iloc[i]["o"]
            current_candle_high = df.iloc[i]["h"]
            current_candle_low = df.iloc[i]["l"]
            current_candle_close = df.iloc[i]["c"]
            current_moving_average_price = df.iloc[i]["MA10"]
    
            if not in_trade:
                if df.iloc[i]["Buy_Signal"] == 1:
                    if not already_signal_exists:
                        # Fresh Buy Signal
    
                        # print(df.iloc[i])
                        # print('Fresh Buy Signal')
                        signal_entry_price = current_candle_high
                        # signal_initial_sl = current_candle_low
                        signal_initial_sl = min(current_candle_low, df.iloc[i-1]['l'], df.iloc[i-2]['l'])
                        # signal_initial_sl = current_candle_low
                        signal_target = (target * (signal_entry_price - signal_initial_sl)) + signal_entry_price
                        signal_creation_time = df.iloc[i]["datetime"]
                        already_signal_exists = True
    
                        # if (signal_entry_price - signal_initial_sl > 400):
                        #     # Skip Signal Candle Due To Big Size
    
                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0
    
                    else:
                        # if current_candle_open > signal_entry_price:
                        #     # Gap Up Open, SKIP trade
    
                        #     # print(df.iloc[i])
                        #     # print('Gap Up Open, Skip Trade')
                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0
    
                        if current_candle_high < signal_entry_price:
                            # Better Candle
    
                            # print(df.iloc[i])
                            # print('Better Candle')
                            signal_entry_price = current_candle_high
                            signal_initial_sl = min(current_candle_low, df.iloc[i-1]['l'], df.iloc[i-2]['l'])
                            signal_target = (target * (signal_entry_price - signal_initial_sl)) + signal_entry_price
                            # signal_initial_sl = current_candle_low
                            signal_creation_time = df.iloc[i]["datetime"]
    
                            # if (signal_entry_price - signal_initial_sl > 400):
                            #     # Skip Signal Candle Due To Big Size
    
                            #     already_signal_exists = False
                            #     signal_entry_price = 100000
                            #     signal_initial_sl = 0
    
                        elif current_candle_high >= signal_entry_price:
                            # Entry Triggered
    
                            # print(df.iloc[i])
                            # print('Entry Triggered')
                            in_trade = True
                            entry_time = df.iloc[i]["datetime"]
                            entry_price = signal_entry_price
                            points = 0
    
                else:
                    if df.iloc[i - 1]["Buy_Signal"] == 1:
                        # Considering the forward bias condition as well now
                        signal_entry_price = df.iloc[i - 1]["h"]
                        signal_initial_sl = min(df.iloc[i-1]['l'], df.iloc[i-2]['l'], df.iloc[i-3]['l'])
                        # signal_initial_sl = df.iloc[i-1]['l']
                        signal_target = (target * (signal_entry_price - signal_initial_sl)) + signal_entry_price
                        signal_creation_time = df.iloc[i - 1]["datetime"]
                        already_signal_exists = True
    
                        if current_candle_high >= signal_entry_price:
                            in_trade = True
                            entry_time = df.iloc[i]["datetime"]
                            entry_price = signal_entry_price
                            points = 0
                        else:
                            # Discard Existing Signal
                            already_signal_exists = False
                            signal_entry_price = 100000
                            signal_initial_sl = 0
    
            if in_trade:
                trade_entry_price = signal_entry_price
                trade_initial_sl = signal_initial_sl
                trade_final_sl = signal_initial_sl
                trade_target = signal_target
    
                if (
                    not is_trailing_active
                    and current_candle_low > current_moving_average_price
                ):
                    is_trailing_active = True
    
                if not is_trailing_active:
                    # if current_candle_open < trade_initial_sl:
                    #     # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     #     print(df.iloc[i])
                    #     #     print('Initial SL Hit Before Gap Down')
                    #     #     in_trade = False
                    #     #     points = trade_initial_sl - trade_entry_price
                    #     #     exit_price = trade_initial_sl
                    #     #     exit_time = df.iloc[i-1]['datetime']
                    #     #     remark = 'Initial SL Hit Before Gap Down'
                    #     # else:
                    #     if (
                    #         df.iloc[i]["datetime"].date() == entry_time.date()
                    #         and df.iloc[i]["datetime"].time() == entry_time.time()
                    #     ):
                    #         if current_candle_close <= trade_initial_sl:
                    #             in_trade = False
                    #             points = trade_initial_sl - trade_entry_price
                    #             exit_price = trade_initial_sl
                    #             exit_time = df.iloc[i]["datetime"]
                    #             remark = "Initial SL hit"
    
                    #     else:
                    #         # Gap Open Outside ISL
    
                    #         # print(df.iloc[i])
                    #         # print('Gap Open Outside ISL')
                    #         in_trade = False
                    #         points = current_candle_open - trade_entry_price
                    #         exit_price = current_candle_open
                    #         exit_time = df.iloc[i]["datetime"]
                    #         remark = "Gap Open Outside ISL"
    
                    if current_candle_low <= trade_initial_sl:
                        # Initial SL Hit
    
                        # print(df.iloc[i])
                        # print('Initial SL Hit')
                        in_trade = False
                        points = trade_initial_sl - trade_entry_price
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL Hit"
    
                    elif current_candle_high >= trade_target:
                        # Target Hit
    
                        # print(df.iloc[i])
                        # print('Target Hit')
                        in_trade = False
                        points = trade_target - trade_entry_price
                        exit_price = trade_target
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Target Hit"
    
                    elif df.iloc[i]["datetime"].time() >= dt.time(15, 20) and not eod_exit_flag:
                        # EOD Exit
    
                        # print(df.iloc[i])
                        # print('EOD Exit')
                        in_trade = False
                        points = current_candle_close - trade_entry_price
                        exit_price = current_candle_close
                        exit_time = df.iloc[i]["datetime"]
                        remark = "EOD Exit"
                        eod_exit_flag = True
    
                else:
                    trade_final_sl = max(trade_initial_sl, current_moving_average_price)
    
                    # if current_candle_open <= trade_initial_sl:
                    #     # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     #     print(df.iloc[i])
                    #     #     print('Initial SL Hit Before Gap Down')
                    #     #     in_trade = False
                    #     #     points = trade_initial_sl - trade_entry_price
                    #     #     exit_price = trade_initial_sl
                    #     #     exit_time = df.iloc[i-1]['datetime']
                    #     #     remark = 'Initial SL Hit'
                    #     # else:
                    #     if (
                    #         df.iloc[i]["datetime"].date() == entry_time.date()
                    #         and df.iloc[i]["datetime"].time() == entry_time.time()
                    #     ):
                    #         if current_candle_close <= trade_initial_sl:
                    #             in_trade = False
                    #             points = trade_initial_sl - trade_entry_price
                    #             exit_price = trade_initial_sl
                    #             exit_time = df.iloc[i]["datetime"]
                    #             remark = "Initial SL hit"
    
                    #     else:
                    #         # Gap Open Outside ISL
    
                    #         # print(df.iloc[i])
                    #         # print('Gap Open Outside ISL')
                    #         in_trade = False
                    #         points = current_candle_open - trade_entry_price
                    #         exit_price = current_candle_open
                    #         exit_time = df.iloc[i]["datetime"]
                    #         remark = "Gap Open Outside ISL"
    
                    if current_candle_low <= trade_initial_sl:
                        # if trade_initial_sl >= current_moving_average_price:
    
                        # Despite Trailing, Initial SL hit
                        # print(df.iloc[i])
                        # print('Initial SL Hit')
                        in_trade = False
                        points = trade_initial_sl - trade_entry_price
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL hit"
    
                    elif current_candle_high >= trade_target:
                        # Target Hit
    
                        # print(df.iloc[i])
                        # print('Target Hit')
                        in_trade = False
                        points = trade_target - trade_entry_price
                        exit_price = trade_target
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Target Hit"
    
                    elif current_candle_close <= trade_final_sl:
                        # Price Closed below TSL i.e. MA10 , TSL Hit
    
                        # print(df.iloc[i])
                        # print('Initial SL Hit')
                        in_trade = False
                        points = current_candle_close - trade_entry_price
                        exit_price = current_candle_close
                        exit_time = df.iloc[i]["datetime"]
                        is_trailing_active = False
                        remark = "TSL Hit"
    
                    elif df.iloc[i]["datetime"].time() >= dt.time(15, 20) and not eod_exit_flag:
                        # EOD Exit
    
                        # print(df.iloc[i])
                        # print('EOD Exit')
                        in_trade = False
                        points = current_candle_close - trade_entry_price
                        exit_price = current_candle_close
                        exit_time = df.iloc[i]["datetime"]
                        remark = "EOD Exit"
                        eod_exit_flag = True
    
                # if points and (entry_price <= 0.1 * (strike/100)):
                if points:
                    if (entry_price - trade_initial_sl > 0) and (entry_time.time() <= dt.time(15, 10)): # To avoid inf qty
                        
                        # qty = int(round(portfolio_value * 5 / entry_price / 15)) * 15
                        qty = 0.01 * PORTFOLIO_VALUE / (abs(entry_price - trade_initial_sl))
                        slippage = 0.015 * (entry_price + exit_price)
                        # slippage = 10
                        final_points = points - slippage
                        # final_points = points
                        trade = {
                            "Index": index,
                            "Strike": strike,
                            "Option Type": asset_class,
                            "Expiry": expiry,
                            "Signal Generated At": signal_creation_time,
                            "Trade Type": "BUY",
                            "Entry Time": entry_time,
                            "Entry Price": entry_price,
                            "Initial SL": trade_initial_sl,
                            "Final SL": trade_final_sl,
                            "Exit Time": exit_time,
                            "Exit Price": exit_price,
                            "Points Captured": points,
                            "Slippages": slippage,
                            "After Costs": final_points,
                            "PnL": final_points * qty,
                            "Remarks": remark,
                            "Qty": qty,
                            # "Leverage": "5x",
                            "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                            "Trade Year": entry_time.year,
                            "Trade Month": entry_time.month,
                        }
                        # print(trade)
                        trade_book.append(trade)
                        
                    points = 0
                    in_trade = False
                    already_signal_exists = False
                    remark = ""
                    is_trailing_active = False
                    eod_exit_flag = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [78]:
async def trade_mtrend_option_selling(
    df, index, strike, asset_class, expiry, starting_index
):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""
    print(df.iloc[-1])
    for i in range(starting_index, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_moving_average_price = df.iloc[i]["MA10"]

        subset_df = df[:i+1]
        if not in_trade:
            if df.iloc[i]["Sell_Signal"] == 1:
                if not already_signal_exists:
                    # Fresh Buy Signal

                    # print(df.iloc[i])
                    print('Fresh Buy Signal')
                    print(subset_df.to_string())
                    
                    signal_entry_price = current_candle_low
                    signal_initial_sl = subset_df['h'].max()
                    signal_creation_time = df.iloc[i]["datetime"]
                    already_signal_exists = True
                    print(signal_entry_price, signal_initial_sl)

                    # if (signal_entry_price - signal_initial_sl > 400):
                    #     # Skip Signal Candle Due To Big Size

                    #     already_signal_exists = False
                    #     signal_entry_price = 100000
                    #     signal_initial_sl = 0

                else:
                    # if current_candle_open < signal_entry_price:
                    #     # Gap Up Open, SKIP trade

                    #     # print(df.iloc[i])
                    #     # print('Gap Up Open, Skip Trade')
                    #     already_signal_exists = False
                    #     signal_entry_price = 100000
                    #     signal_initial_sl = 0

                    if current_candle_low > signal_entry_price:
                        # Better Candle

                        # print(df.iloc[i])
                        print('Better Candle')
                        print(subset_df.to_string())
                        
                        signal_entry_price = current_candle_low
                        signal_initial_sl = subset_df['h'].max()
                        signal_creation_time = df.iloc[i]["datetime"]
                        print(signal_entry_price, signal_initial_sl)

                        # if (signal_entry_price - signal_initial_sl > 400):
                        #     # Skip Signal Candle Due To Big Size

                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0

                    elif current_candle_low <= signal_entry_price:
                        # Entry Triggered

                        # print(df.iloc[i])
                        print('Entry Triggered')
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        print(entry_price, signal_initial_sl)
                        points = 0

            else:
                if (df.iloc[i - 1]["Sell_Signal"] == 1) and (df.iloc[i-1]['datetime'].time() != dt.time(15, 15)):
                    subset_df = df[:i-1]
                    print(subset_df.to_string())
                    print(df.iloc[i-1])
                    # Considering the forward bias condition as well now
                    print('Forward Bias Avoided Condition Triggered')
                    signal_entry_price = df.iloc[i - 1]["l"]
                    signal_initial_sl = subset_df['h'].max()
                    signal_creation_time = df.iloc[i - 1]["datetime"]
                    already_signal_exists = True
                    print(signal_entry_price, signal_initial_sl)
                    # print(subset_df.to_string())

                    if current_candle_low <= signal_entry_price:
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
                        print(entry_price, signal_initial_sl)
                    else:
                        # Discard Existing Signal
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0

        if in_trade:
            print(df.iloc[i]['datetime'])
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl

            # if (
            #     not is_trailing_active
            #     and current_candle_high < current_moving_average_price
            # ):
            #     is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open > trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit Before Gap Down'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close >= trade_initial_sl:
                            in_trade = False
                            points = -1 * (trade_initial_sl - trade_entry_price)
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL

                        # print(df.iloc[i])
                        print('Gap Open Outside ISL')
                        in_trade = False
                        points = trade_entry_price - current_candle_open
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_high >= trade_initial_sl:
                    # Initial SL Hit

                    print(df.iloc[i])
                    print('Initial SL Hit')
                    print(current_candle_high, trade_initial_sl)
                    in_trade = False
                    points = -1 * (trade_initial_sl - trade_entry_price)
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif df.iloc[i]["datetime"].time() == dt.time(15, 15):
                    # EOD Exit

                    # print(df.iloc[i])
                    print('EOD Exit')
                    in_trade = False
                    points = trade_entry_price - current_candle_close
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "EOD Exit"

            else:
                trade_final_sl = min(trade_initial_sl, current_moving_average_price)

                if current_candle_open > trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close >= trade_initial_sl:
                            in_trade = False
                            points = -1 * (trade_initial_sl - trade_entry_price)
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL

                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = trade_entry_price - current_candle_open
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_high >= trade_initial_sl:
                    # if trade_initial_sl >= current_moving_average_price:

                    # Despite Trailing, Initial SL hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = -1 * (trade_initial_sl - trade_entry_price)
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif current_candle_close >= trade_final_sl:
                    # Price Closed below TSL i.e. MA10 , TSL Hit

                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = -1 * (current_candle_close - trade_entry_price)
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

                elif df.iloc[i]["datetime"].time() == dt.time(15, 15):
                    # EOD Exit

                    # print(df.iloc[i])
                    # print('EOD Exit')
                    in_trade = False
                    points = trade_entry_price - current_candle_close
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "EOD Exit"

            if points:
                # qty = int(round(portfolio_value * 5 / entry_price / 15)) * 15
                qty = int(round(PORTFOLIO_VALUE * INDEX_LEV / strike / 15)) * 15
                slippage = 0.01 * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Index": index,
                    "Strike": strike,
                    "Option Type": asset_class,
                    "Expiry": expiry,
                    "Signal Generated At": signal_creation_time,
                    "Trade Type": "SELL",
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "Slippages": slippage,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    # "Leverage": "5x",
                    "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                # print(trade)
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [79]:
async def trade_mtrend_options_2(
    df, index, strike, asset_class, expiry, starting_index
):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    signal_target = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""

    for i in range(starting_index, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_moving_average_price = df.iloc[i]["MA10"]

        if not in_trade:
            if df.iloc[i]["Buy_Signal"] == 1:
                if not already_signal_exists:
                    # Fresh Buy Signal

                    # print(df.iloc[i])
                    # print('Fresh Buy Signal')
                    signal_entry_price = current_candle_high
                    signal_initial_sl = current_candle_low
                    signal_target = (
                        3 * (current_candle_high - current_candle_low)
                    ) + current_candle_high
                    signal_creation_time = df.iloc[i]["datetime"]
                    already_signal_exists = True

                    # if (signal_entry_price - signal_initial_sl > 400):
                    #     # Skip Signal Candle Due To Big Size

                    #     already_signal_exists = False
                    #     signal_entry_price = 100000
                    #     signal_initial_sl = 0

                else:
                    if current_candle_open > signal_entry_price:
                        # Gap Up Open, SKIP trade

                        # print(df.iloc[i])
                        # print('Gap Up Open, Skip Trade')
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0

                    elif current_candle_high < signal_entry_price:
                        # Better Candle

                        # print(df.iloc[i])
                        # print('Better Candle')
                        signal_entry_price = current_candle_high
                        signal_initial_sl = current_candle_low
                        signal_target = (
                            3 * (current_candle_high - current_candle_low)
                        ) + current_candle_high
                        signal_creation_time = df.iloc[i]["datetime"]

                        # if (signal_entry_price - signal_initial_sl > 400):
                        #     # Skip Signal Candle Due To Big Size

                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0

                    elif current_candle_high >= signal_entry_price:
                        # Entry Triggered

                        # print(df.iloc[i])
                        # print('Entry Triggered')
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0

            else:
                if df.iloc[i - 1]["Buy_Signal"] == 1:
                    # Considering the forward bias condition as well now
                    signal_entry_price = df.iloc[i - 1]["h"]
                    signal_initial_sl = df.iloc[i - 1]["l"]
                    signal_target = (
                        3 * (current_candle_high - current_candle_low)
                    ) + current_candle_high
                    signal_creation_time = df.iloc[i - 1]["datetime"]
                    already_signal_exists = True

                    if current_candle_high >= signal_entry_price:
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
                    else:
                        # Discard Existing Signal
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl

            if (
                not is_trailing_active
                and current_candle_low > current_moving_average_price
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open < trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit Before Gap Down'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL

                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Initial SL Hit

                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif current_candle_high >= signal_target:
                    # Target Hit

                    # print(df.iloc[i])
                    # print('Target Hit')
                    in_trade = False
                    points = signal_target - trade_entry_price
                    exit_price = signal_target
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Target Hit"

                elif df.iloc[i]["datetime"].time() == dt.time(15, 20):
                    # EOD Exit

                    # print(df.iloc[i])
                    # print('EOD Exit')
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "EOD Exit"

            else:
                trade_final_sl = max(trade_initial_sl, current_moving_average_price)

                if current_candle_open <= trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL

                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # if trade_initial_sl >= current_moving_average_price:

                    # Despite Trailing, Initial SL hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif current_candle_high >= signal_target:
                    # Target Hit

                    # print(df.iloc[i])
                    # print('Target Hit')
                    in_trade = False
                    points = signal_target - trade_entry_price
                    exit_price = signal_target
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Target Hit"

                elif current_candle_close <= trade_final_sl:
                    # Price Closed below TSL i.e. MA10 , TSL Hit

                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

                elif df.iloc[i]["datetime"].time() == dt.time(15, 20):
                    # EOD Exit

                    # print(df.iloc[i])
                    # print('EOD Exit')
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "EOD Exit"

            if points:
                # qty = int(round(portfolio_value * 5 / entry_price / 15)) * 15
                qty = 0.01 * PORTFOLIO_VALUE / (abs(entry_price - trade_initial_sl))
                slippage = 0.005 * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Index": index,
                    "Strike": strike,
                    "Option Type": asset_class,
                    "Expiry": expiry,
                    "Signal Generated At": signal_creation_time,
                    "Trade Type": "BUY",
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Initial Target": signal_target,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "Slippages": slippage,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    # "Leverage": "5x",
                    "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [80]:
# async def trade(df):

#     start_date = dt.date(2024, 1, 1)
#     end_date = dt.date(2024, 1, 31)

#     current_date = start_date

#     combined_trades = pd.DataFrame()
#     total_trades = pd.DataFrame()

#     while current_date <= end_date:
#         if current_date in trading_days_set:

#             starting_time = dt.time(9, 15)
#             time_of_day = dt.time(14, 30)
#             ending_time = dt.time(15, 30)

#             search_datetime = dt.datetime.combine(current_date, time_of_day)

#             spot_open = df.loc[df["datetime"] >= search_datetime, "open"].iloc[0]
#             # print(spot_open)

#             spot_atm = int(round(spot_open / 100) * 100)
#             # print(spot_atm)

#             nearest_expiry = await get_expiry_finnifty(current_date)
#             # print(nearest_expiry)

#             # if current_date == nearest_expiry:
#             print(f"Current Date : {current_date}")

#             ce_df = await fetch_data(
#                 index=INDEX,
#                 start_date=current_date,
#                 start_time=starting_time,
#                 end_date=current_date,
#                 end_time=ending_time,
#                 strike=spot_atm,
#                 asset_class="C",
#                 expiry=nearest_expiry,
#             )
#             if not isinstance(ce_df, str):
#                 # print("IF CONDITION TRIGGERED -> CE Option")
#                 # ce_df = ce_df.rename(
#                 #     {"open": "o", "high": "h", "low": "l", "close": "c"}
#                 # )
#                 ce_df = ce_df.select(["datetime", "o", "h", "l", "c"])
#                 ce_df = resample(ce_df, "5m")
#                 ce_df_pandas = ce_df.to_pandas()
#                 ce_df = generate_signals1(ce_df_pandas)
#                 # ce_df = generate_signals2(ce_df_pandas)
#                 # print(ce_df.tail())
#             else:
#                 # print("ELSE CONDITION TRIGGERED -> CE Option")
#                 # print(ce_df)
#                 current_date += dt.timedelta(days=1)
#                 continue

#             pe_df = await fetch_data(
#                 index=INDEX,
#                 start_date=current_date,
#                 start_time=starting_time,
#                 end_date=current_date,
#                 end_time=ending_time,
#                 strike=spot_atm,
#                 asset_class="P",
#                 expiry=nearest_expiry,
#             )
#             if not isinstance(pe_df, str):
#                 # print("IF CONDITION TRIGGERED -> PE Option")
#                 # pe_df = pe_df.rename(
#                 #     {"open": "o", "high": "h", "low": "l", "close": "c"}
#                 # )
#                 pe_df = pe_df.select(["datetime", "o", "h", "l", "c"])
#                 pe_df = resample(pe_df, "5m")
#                 pe_df_pandas = pe_df.to_pandas()
#                 pe_df = generate_signals1(pe_df_pandas)
#                 # pe_df = generate_signals2(pe_df_pandas)
#                 # print(pe_df.tail())
#             else:
#                 # print("ELSE CONDITION TRIGGERED -> PE Option")
#                 # print(pe_df)
#                 current_date += dt.timedelta(days=1)
#                 continue

#             # print(
#             #     ce_df_pandas.to_string(), "\n", pe_df_pandas.to_string()
#             # )

#             # print('WORKING ON CE TRADES')
#             ce_trades = await trade_mtrend_option_buying(
#                 df=ce_df_pandas,
#                 index=INDEX,
#                 strike=spot_atm,
#                 asset_class="C",
#                 expiry=nearest_expiry,
#                 search_datetime=search_datetime,
#                 # starting_index=starting_index,
#             )
#             # print('WORKING ON PE TRADES')
#             pe_trades = await trade_mtrend_option_buying(
#                 df=pe_df_pandas,
#                 index=INDEX,
#                 strike=spot_atm,
#                 asset_class="P",
#                 expiry=nearest_expiry,
#                 search_datetime=search_datetime,
#                 # starting_index=starting_index,
#             )
#             combined_trades = pd.concat([combined_trades, ce_trades], ignore_index=True)
#             combined_trades = pd.concat([combined_trades, pe_trades], ignore_index=True)
#             # print(combined_trades)

#         current_date += dt.timedelta(days=1)

#     total_trades = pd.concat([total_trades, combined_trades], ignore_index=True)

#     return total_trades

# # tb = await trade(data)
# # tb

In [81]:
async def trade_expiry(df, signal_ma, trailing_ma, time_of_day, tf, num_candles, ofs, target):

    start_date = dt.date(2022, 10, 1)
    end_date = dt.date(2024, 7, 31)

    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()

    while current_date <= end_date:
        if current_date in trading_days_set:

            starting_time = dt.time(9, 15)
            # time_of_day = dt.time(13, 0)
            ending_time = dt.time(15, 30)

            search_datetime = dt.datetime.combine(current_date, time_of_day)

            spot_open = df.loc[df["datetime"] >= search_datetime, "open"].iloc[0]
            # print(df.loc[df['datetime'] >= search_datetime, 'datetime'].iloc[0])
            # print(spot_open)

            spot_atm = int(round(spot_open / 50) * 50)
            # print(spot_atm)

            nearest_expiry = await get_expiry_finnifty(current_date)
            # print(nearest_expiry)

            if current_date == nearest_expiry:
                # print(f"Current Date : {current_date}")

                ce_df = await fetch_data(
                    index=INDEX,
                    start_date=current_date,
                    start_time=starting_time,
                    end_date=current_date,
                    end_time=ending_time,
                    strike=spot_atm,
                    asset_class="C",
                    expiry=nearest_expiry,
                )
                if not isinstance(ce_df, str) and (ce_df is not None):
                    # print("IF CONDITION TRIGGERED -> CE Option")
                    # ce_df = ce_df.rename(
                    #     {"open": "o", "high": "h", "low": "l", "close": "c"}
                    # )
                    ce_df = ce_df.select(["datetime", "o", "h", "l", "c"])
                    ce_df = resample(ce_df, tf, ofs)
                    ce_df_pandas = ce_df.to_pandas()
                    ce_df = generate_signals1(ce_df_pandas, signal_ma, trailing_ma, num_candles)
                    # ce_df = generate_signals2(ce_df_pandas)
                    # print(ce_df.tail())
                else:
                    # print("ELSE CONDITION TRIGGERED -> CE Option")
                    # print(ce_df)
                    current_date += dt.timedelta(days=1)
                    continue
    
                pe_df = await fetch_data(
                    index=INDEX,
                    start_date=current_date,
                    start_time=starting_time,
                    end_date=current_date,
                    end_time=ending_time,
                    strike=spot_atm,
                    asset_class="P",
                    expiry=nearest_expiry,
                )
                if not isinstance(pe_df, str) and (pe_df is not None):
                    # print("IF CONDITION TRIGGERED -> PE Option")
                    # pe_df = pe_df.rename(
                    #     {"open": "o", "high": "h", "low": "l", "close": "c"}
                    # )
                    pe_df = pe_df.select(["datetime", "o", "h", "l", "c"])
                    pe_df = resample(pe_df, tf, ofs)
                    pe_df_pandas = pe_df.to_pandas()
                    pe_df = generate_signals1(pe_df_pandas, signal_ma, trailing_ma, num_candles)
                    # pe_df = generate_signals2(pe_df_pandas)
                    # print(pe_df.tail())
                else:
                    # print("ELSE CONDITION TRIGGERED -> PE Option")
                    # print(pe_df)
                    current_date += dt.timedelta(days=1)
                    continue
    
                # print(
                #     ce_df_pandas.to_string(), "\n", pe_df_pandas.to_string()
                # )
    
                # print('WORKING ON CE TRADES')
                ce_trades = await trade_mtrend_option_buying(
                    df=ce_df_pandas,
                    index=INDEX,
                    strike=spot_atm,
                    asset_class="C",
                    expiry=nearest_expiry,
                    search_datetime=search_datetime,
                    target=target,
                    # starting_index=starting_index,
                )
                # print('WORKING ON PE TRADES')
                pe_trades = await trade_mtrend_option_buying(
                    df=pe_df_pandas,
                    index=INDEX,
                    strike=spot_atm,
                    asset_class="P",
                    expiry=nearest_expiry,
                    search_datetime=seaw
                    rch_datetime,
                    target=target,
                    # starting_index=starting_index,
                )
                combined_trades = pd.concat([combined_trades, ce_trades], ignore_index=True)
                combined_trades = pd.concat([combined_trades, pe_trades], ignore_index=True)
                # print(combined_trades)

        current_date += dt.timedelta(days=1)

    total_trades = pd.concat([total_trades, combined_trades], ignore_index=True)

    return total_trades

# tb = await trade_expiry(data)
# tb

In [82]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [83]:
# stats_expiry = generate_stats(tb, i, j, k)

# tb_ce = tb[tb['Option Type'] == 'C']
# tb_pe = tb[tb['Option Type'] == 'P']
# stats_ce = generate_stats(tb_ce, i, j, k)
# stats_pe = generate_stats(tb_pe, i, j, k)

# for overall_roi_dd_ratio, stats_df in stats_expiry.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Overall Stats Combined')
#             print(stats_df.to_string())

# for overall_roi_dd_ratio, stats_df in stats_ce.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Only CE')
#             print(stats_df.to_string())

# for overall_roi_dd_ratio, stats_df in stats_pe.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Only PE')
#             print(stats_df.to_string())

In [84]:
# tb_expiry = tb
# tb_ce = tb[tb['Option Type'] == 'C']
# tb_pe = tb[tb['Option Type'] == 'P']
# stats_expiry = generate_stats(tb_expiry, i, j, k)
# stats_ce = generate_stats(tb_ce, i, j, k)
# stats_pe = generate_stats(tb_pe, i, j, k)
# for overall_roi_dd_ratio, stats_df in stats_expiry.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Overall Stats Combined')
#             print(stats_df.to_string())

# for overall_roi_dd_ratio, stats_df in stats_ce.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Only CE')
#             print(stats_df.to_string())

# for overall_roi_dd_ratio, stats_df in stats_pe.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Only PE')
#             print(stats_df.to_string())

In [85]:
tb_ce

,Index,Strike,Option Type,Expiry,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,Slippages,After Costs,PnL,Remarks,Qty,ROI%,Trade Year,Trade Month
0,bnf,18100,C,2017-01-05,2017-01-05 14:55:00,BUY,2017-01-05 15:00:00,44.4000,23.6000,27.8640,2017-01-05 15:10:00,23.6000,-20.8000,1.0200,-21.8200,-10490.3846,Initial SL hit,480.7692,-1.0490,2017,1
1,bnf,18800,C,2017-01-12,2017-01-12 14:55:00,BUY,2017-01-12 15:00:00,77.0000,54.3000,72.8860,2017-01-12 15:10:00,72.0000,-5.0000,2.2350,-7.2350,-3187.2247,TSL Hit,440.5286,-0.3187,2017,1
2,bnf,19300,C,2017-01-25,2017-01-25 14:30:00,BUY,2017-01-25 14:35:00,19.8000,7.8000,42.3940,2017-01-25 15:20:00,161.0000,141.2000,2.7120,138.4880,115406.6667,EOD Exit,833.3333,11.5407,2017,1
3,bnf,20100,C,2017-02-02,2017-02-02 14:50:00,BUY,2017-02-02 14:55:00,27.8000,18.8500,18.8500,2017-02-02 14:55:00,18.8500,-8.9500,0.6998,-9.6497,-10781.8436,Initial SL Hit,1117.3184,-1.0782,2017,2
4,bnf,20100,C,2017-02-09,2017-02-09 14:45:00,BUY,2017-02-09 14:50:00,58.8000,39.0000,39.0000,2017-02-09 14:55:00,39.0000,-19.8000,1.4670,-21.2670,-10740.9091,Initial SL Hit,505.0505,-1.0741,2017,2
5,bnf,20900,C,2017-02-23,2017-02-23 14:50:00,BUY,2017-02-23 14:55:00,89.1000,62.9500,62.9500,2017-02-23 14:55:00,62.9500,-26.1500,2.2808,-28.4307,-10872.1797,Initial SL Hit,382.4092,-1.0872,2017,2
8,bnf,22300,C,2017-04-27,2017-04-27 14:40:00,BUY,2017-04-27 14:45:00,41.4000,29.3000,29.3000,2017-04-27 14:45:00,29.3000,-12.1000,1.0605,-13.1605,-10876.4463,Initial SL Hit,826.4463,-1.0876,2017,4
9,bnf,22700,C,2017-05-04,2017-05-04 14:45:00,BUY,2017-05-04 14:50:00,28.7500,16.3500,16.3500,2017-05-04 14:55:00,16.3500,-12.4000,0.6765,-13.0765,-10545.5645,Initial SL hit,806.4516,-1.0546,2017,5
12,bnf,23000,C,2017-05-25,2017-05-25 14:30:00,BUY,2017-05-25 14:35:00,93.0000,56.6000,80.3360,2017-05-25 15:20:00,184.0000,91.0000,4.1550,86.8450,23858.5165,EOD Exit,274.7253,2.3859,2017,5
19,bnf,24200,C,2017-10-12,2017-10-12 14:40:00,BUY,2017-10-12 14:45:00,29.4500,21.1000,45.3840,2017-10-12 15:20:00,152.0000,122.5500,2.7217,119.8282,143506.8862,EOD Exit,1197.6048,14.3507,2017,10


In [28]:
# tb.to_csv('MIDCP_5m_40_10_1330_3sl_1c.csv')
# tb.to_csv('MIDCP.csv')

In [29]:
# tb_targets = tb[tb['Remarks'] == 'Target Hit']
# print(len(tb_targets))
# print(tb_targets['ROI%'].sum())

In [30]:
# tb_targets

In [31]:
# tb_expiry = tb
# tb_ce = tb[tb['Option Type'] == 'C']
# tb_pe = tb[tb['Option Type'] == 'P']
# stats_expiry = generate_stats(tb_expiry, i, j, k)
# stats_ce = generate_stats(tb_ce, i, j, k)
# stats_pe = generate_stats(tb_pe, i, j, k)
# for overall_roi_dd_ratio, stats_df in stats_expiry.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Overall Stats Combined')
#             print(stats_df.to_string())

# for overall_roi_dd_ratio, stats_df in stats_ce.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Only CE')
#             print(stats_df.to_string())

# for overall_roi_dd_ratio, stats_df in stats_pe.items():
#         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
#             # stats_dict[overall_roi_dd_ratio] = stats_df
#             # Print the stats
#             print('Only PE')
#             print(stats_df.to_string())

In [86]:
stats_dict = {}
tfs = ['3m', '5m']
times = [dt.time(13, 30), dt.time(14, 0)]
targets = [3, 6, 10, 15, 20, 500]
# times = [dt.time(12, 0)]
for i in range(5, 31, 5):
    for j in range(5, 21, 5):
        for k in times:
            for x in  range(1, 4):
                for tf in tfs:
                    for target in targets:
                        if tf == '10m':
                            ofs = '5m'
                        else:
                            ofs = '0m'
                        variation = f'SMA {i}, TMA {j}, Time {k}, Num Candles {x}, TF {tf}, Target: {target}R'
                        print(variation)
                        tb = await trade_expiry(data, i, j, k, tf, x, ofs, target)
                        if not isinstance(tb, str):
                            # print(tb)
                            tb_expiry = tb
                            tb_ce = tb[tb['Option Type'] == 'C']
                            tb_pe = tb[tb['Option Type'] == 'P']
                            stats_expiry = generate_stats(tb_expiry, variation)
                            stats_ce = generate_stats(tb_ce, variation)
                            stats_pe = generate_stats(tb_pe, variation)
                            for overall_roi_dd_ratio, stats_df in stats_expiry.items():
                                    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 12:
                                        stats_dict[overall_roi_dd_ratio] = stats_df
                                        # Print the stats
                                        print('Overall Stats Combined')
                                        print(stats_df.to_string())
            
                            # for overall_roi_dd_ratio, stats_df in stats_ce.items():
                            #         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 5:
                            #             stats_dict[overall_roi_dd_ratio] = stats_df
                            #             # Print the stats
                            #             print('Only CE')
                            #             print(stats_df.to_string())
            
                            # for overall_roi_dd_ratio, stats_df in stats_pe.items():
                            #         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 5:
                            #             stats_dict[overall_roi_dd_ratio] = stats_df
                            #             # Print the stats
                            #             print('Only PE')
                            #             print(stats_df.to_string())
            
sorted_stats = {k: v for k, v in sorted(stats_dict.items(), key=lambda item: item[0], reverse=True)}

SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 3m, Target: 3R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 3m, Target: 6R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 3m, Target: 10R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 3m, Target: 15R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 3m, Target: 20R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 3m, Target: 500R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 5m, Target: 3R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 5m, Target: 6R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 5m, Target: 10R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 5m, Target: 15R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 5m, Target: 20R
SMA 5, TMA 5, Time 13:30:00, Num Candles 1, TF 5m, Target: 500R
SMA 5, TMA 5, Time 13:30:00, Num Candles 2, TF 3m, Target: 3R
SMA 5, TMA 5, Time 13:30:00, Num Candles 2, TF 3m, Target: 6R
SMA 5, TMA 5, Time 13:30:00, Num Candles 2, TF 3m, Target: 10R
SMA 5, TMA 5, Time 13:30:00, Num Candles 2, TF 3m, Target: 

In [88]:
# sorted_stats
for roi, stats_ in sorted_stats.items():
    print(roi)
    print(stats_.to_string())

In [89]:
sorted_stats

{}

In [ ]:
tb_check = await trade_expiry(data, 30, 4, dt.time(14, 0), "5m", 4, "0m", 100)
stats_check = generate_stats(tb_check, "...")

In [61]:
first_key = list(stats_check.keys())[0]  # Get the 0th key
print(stats_check[first_key].to_string())

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2017      77.5294           31  45.1613                6.6876             -0.9469      -3.2424      23.9112       ...
2018      19.1858           50  34.0000                2.9294             -0.9277     -10.0615       1.9069       ...
2019      27.7700           40  37.5000                3.2970             -0.8674      -7.0023       3.9659       ...
2020       9.5160           46  36.9565                2.1477             -0.9309      -8.6902       1.0950       ...
2021      27.6585           41  43.9024                2.6795             -0.8945      -3.7881       7.3014       ...
2022      26.3666           44  45.4545                2.4454             -0.9392      -8.4003       3.1388       ...
2023       5.2808           52  30.7692                2.1624             -0.8144     -11.0994       0.4758       ...
2024      13.6855            7  28.5714                9

In [ ]:
# df = pd.DataFrame(list(sorted_stats.items()), columns=['Overall ROI/DD', 'Stats'])

# df.to_csv('BNF 5 candles signal 3 candles SL stats.csv', index=False)

In [118]:
def generate_signals1(df, signal_ma, trailing_ma):
    df["c"] = pd.to_numeric(df["c"], errors="coerce")
    df["MA20"] = df["c"].rolling(window=signal_ma).mean()
    df["MA10"] = df["c"].rolling(window=trailing_ma).mean()

    df["Buy_Signal"] = 0

    # Generate signals using boolean masking
    buy_signal_mask = ((
        df["MA20"]
        > df["MA20"].shift(1))
        & (df["MA20"].shift(1) > df["MA20"].shift(2))
        & (df['MA20'].shift(2) > df['MA20'].shift(3))
        & (df['MA20'].shift(3) > df['MA20'].shift(4))
        & (df['MA20'].shift(4) > df['MA20'].shift(5))
    )

    df.loc[buy_signal_mask, "Buy_Signal"] = 1

    return df

In [119]:
stats_dict = {}

times = [dt.time(12, 0), dt.time(13, 0), dt.time(13, 30), dt.time(14, 0), dt.time(14, 30)]
for i in range(6, 65, 2):
    for j in range(4, 41, 2):
        for k in times:
            print(f'Signal MA {i}, Trailing MA {j}, Time of Day {k}')
            tb = await trade_expiry(data, i, j, k)
            if not isinstance(tb, str):
                # print(tb)
                tb_expiry = tb
                tb_ce = tb[tb['Option Type'] == 'C']
                tb_pe = tb[tb['Option Type'] == 'P']
                stats_expiry = generate_stats(tb_expiry, i, j, k)
                stats_ce = generate_stats(tb_ce, i, j, k)
                stats_pe = generate_stats(tb_pe, i, j, k)
                for overall_roi_dd_ratio, stats_df in stats_expiry.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 3:
                            stats_dict[overall_roi_dd_ratio] = stats_df
                            # Print the stats
                            print('Overall Stats Combined')
                            print(stats_df.to_string())

                for overall_roi_dd_ratio, stats_df in stats_ce.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 3:
                            stats_dict[overall_roi_dd_ratio] = stats_df
                            # Print the stats
                            print('Only CE')
                            print(stats_df.to_string())

                for overall_roi_dd_ratio, stats_df in stats_pe.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 3:
                            stats_dict[overall_roi_dd_ratio] = stats_df
                            # Print the stats
                            print('Only PE')
                            print(stats_df.to_string())

sorted_stats = {k: v for k, v in sorted(stats_dict.items(), key=lambda item: item[0], reverse=True)}

Signal MA 6, Trailing MA 4, Time of Day 12:00:00
Signal MA 6, Trailing MA 4, Time of Day 13:00:00


/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA 6, Trailing MA 4, Time of Day 13:30:00
Signal MA 6, Trailing MA 4, Time of Day 14:00:00


/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA 6, Trailing MA 4, Time of Day 14:30:00


...

/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA 64, Trailing MA 40, Time of Day 14:00:00


/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA 64, Trailing MA 40, Time of Day 14:30:00


/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)
/tmp/ipykernel_3486/3231552177.py:70: RuntimeWarning: divide by zero encountered in scalar divide
  overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
/tmp/ipykernel_3486/3231552177.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


In [120]:
for ratio, stats in sorted_stats.items():
    print(ratio)
    print((pd.DataFrame(stats)).to_string())

5.579141899371651
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio         Variation
2023      11.3378            6  33.3333                7.2623             -0.7967      -2.1381       5.3028  28 , 4, 14:30:00
2024       0.5908            1 100.0000                0.5908                 NaN       0.0000          inf  28 , 4, 14:30:00
Overall   11.9286            7  42.8571                5.0385             -0.7967      -2.1381       5.5791  28 , 4, 14:30:00
4.920655291021365
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio         Variation
2023      16.6399           16  25.0000                6.9632             -0.9344      -4.3676       3.8099  54 , 4, 13:00:00
2024      15.2285            4  50.0000                8.6793             -1.0651      -1.0722      14.2032  54 , 4, 13:00:00
Overall   31.8684           20  30.0000                7.5353             -0.9531 

In [117]:
i =28
j = 4
k = dt.time(12, 0)
tb = await trade_expiry(data, i, j, k)
tb_expiry = tb
tb_ce = tb[tb['Option Type'] == 'C']
tb_pe = tb[tb['Option Type'] == 'P']
stats_expiry = generate_stats(tb_expiry, i, j, k)
stats_ce = generate_stats(tb_ce, i, j, k)
stats_pe = generate_stats(tb_pe, i, j, k)
for overall_roi_dd_ratio, stats_df in stats_expiry.items():
    if overall_roi_dd_ratio is not None:
        # stats_dict[overall_roi_dd_ratio] = stats_df
        # Print the stats
        print('Overall Stats Combined')
        print(stats_df.to_string())

for overall_roi_dd_ratio, stats_df in stats_ce.items():
    if overall_roi_dd_ratio is not None:
        # stats_dict[overall_roi_dd_ratio] = stats_df
        # Print the stats
        print('Only CE')
        print(stats_df.to_string())

for overall_roi_dd_ratio, stats_df in stats_pe.items():
    if overall_roi_dd_ratio is not None:
        # stats_dict[overall_roi_dd_ratio] = stats_df
        # Print the stats
        print('Only PE')
        print(stats_df.to_string())

Overall Stats Combined
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio         Variation
2023      11.0905          141  29.0780                2.4830             -0.9071     -19.2398       0.5764  28 , 4, 12:00:00
2024      22.5524           27  18.5185                8.4401             -0.8931     -12.6509       1.7827  28 , 4, 12:00:00
Overall   33.6429          168  27.3810                3.1305             -0.9046     -29.7791       1.1297  28 , 4, 12:00:00
Only CE
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio         Variation
2023      21.1852           45  35.5556                2.8351             -0.8337      -6.4814       3.2686  28 , 4, 12:00:00
2024      35.0426           14  35.7143                8.4401             -0.7953      -3.7465       9.3534  28 , 4, 12:00:00
Overall   56.2278           59  35.5932                4.1697             -0.8246      

In [115]:
tb_ce

,Index,Strike,Option Type,Expiry,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,Slippages,After Costs,PnL,Remarks,Qty,ROI%,Trade Year,Trade Month
6,bnf,41800,C,2023-01-25,2023-01-25 14:05:00,BUY,2023-01-25 14:10:00,156.3500,67.5500,67.5500,2023-01-25 14:30:00,67.5500,-88.8000,3.3585,-92.1585,-25945.5236,Initial SL Hit,281.5315,-1.0378,2023,1
9,bnf,40300,C,2023-02-02,2023-02-02 14:50:00,BUY,2023-02-02 14:55:00,441.7500,195.2500,195.2500,2023-02-02 15:20:00,368.9000,-72.8500,12.1597,-85.0098,-8621.6785,EOD Exit,101.4199,-0.3449,2023,2
11,bnf,41300,C,2023-02-09,2023-02-09 14:00:00,BUY,2023-02-09 14:05:00,289.1000,141.4000,242.8125,2023-02-09 15:20:00,253.6500,-35.4500,8.1412,-43.5913,-7378.3429,EOD Exit,169.2620,-0.2951,2023,2
14,bnf,40000,C,2023-02-23,2023-02-23 13:55:00,BUY,2023-02-23 14:00:00,153.5000,54.2000,54.2000,2023-02-23 14:35:00,54.2000,-99.3000,3.1155,-102.4155,-25784.3656,Initial SL Hit,251.7623,-1.0314,2023,2
21,bnf,39000,C,2023-03-16,2023-03-16 14:20:00,BUY,2023-03-16 14:25:00,358.0000,61.1500,307.4500,2023-03-16 14:30:00,293.1000,-64.9000,9.7665,-74.6665,-6288.2348,TSL Hit,84.2176,-0.2515,2023,3
22,bnf,39000,C,2023-03-16,2023-03-16 14:35:00,BUY,2023-03-16 14:40:00,321.8000,189.5500,189.5500,2023-03-16 14:45:00,189.5500,-132.2500,7.6703,-139.9203,-26449.9527,Initial SL Hit,189.0359,-1.0580,2023,3
23,bnf,39000,C,2023-03-16,2023-03-16 14:55:00,BUY,2023-03-16 15:00:00,136.8000,76.7500,76.7500,2023-03-16 15:00:00,76.7500,-60.0500,3.2033,-63.2533,-26333.5762,Initial SL Hit,416.3197,-1.0533,2023,3
24,bnf,39000,C,2023-03-16,2023-03-16 15:10:00,BUY,2023-03-16 15:15:00,101.7500,58.6000,113.1375,2023-03-16 15:20:00,133.2500,31.5000,3.5250,27.9750,16207.9954,EOD Exit,579.3743,0.6483,2023,3
25,bnf,40100,C,2023-03-23,2023-03-23 12:10:00,BUY,2023-03-23 12:15:00,94.5500,85.0500,137.4375,2023-03-23 12:50:00,124.6000,30.0500,3.2872,26.7627,70428.2895,TSL Hit,2631.5789,2.8171,2023,3
26,bnf,40100,C,2023-03-23,2023-03-23 13:15:00,BUY,2023-03-23 13:20:00,104.6500,85.8000,115.0000,2023-03-23 13:35:00,113.5000,8.8500,3.2723,5.5777,7397.5464,TSL Hit,1326.2599,0.2959,2023,3


In [44]:
tb["Entry Time"] = pd.to_datetime(tb["Entry Time"])
tb_expiry = tb[tb["Entry Time"].dt.date == tb["Expiry"]]
tb_calls = tb[tb['Option Type'] == 'C']
tb_puts = tb[tb['Option Type'] == 'P']
tb_expiry_ce = tb_expiry[tb_expiry['Option Type'] == 'C']
tb_expiry_pe = tb_expiry[tb_expiry['Option Type'] == 'P']
# tb['ROI%'].sum()

In [45]:
tb["ROI%"].sum(), tb_expiry["ROI%"].sum(), tb_expiry_ce['ROI%'].sum() , tb_expiry_pe['ROI%'].sum()

(-77.11243038342835, -77.11243038342835, 2.140734869226865, -79.25316525265521)

In [46]:
stats_df8 = pd.DataFrame(
    index=range(2023, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_expiry
# combined_df_sorted = tb_expiry_ce
# combined_df_sorted = tb_expiry_pe

# Iterate over each year
for year in range(2023, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2023,-59.4867,103,18.4466,5.6486,-1.9858,-69.3946,-0.8572
2024,-17.6257,29,17.2414,5.8774,-1.9589,-30.9789,-0.5690
Overall,-77.1124,132.0000,18.1818,5.6963,-1.9799,-102.5040,-0.7523


# MTrend Option Buying Intraday 0 DTE - MIDCPNIFTY (CE , PE) (40, 6, 2, 13:00 pm)

In [180]:
tb_expiry

,Index,Strike,Option Type,Expiry,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,Slippages,After Costs,PnL,Remarks,Qty,ROI%,Trade Year,Trade Month
17,midcpnifty,9200,C,2023-09-11,2023-09-11 13:10:00,BUY,2023-09-11 13:15:00,7.2000,5.5000,5.5000,2023-09-11 13:40:00,5.5000,-1.7000,0.1905,-1.8905,-55602.9412,Initial SL Hit,29411.7647,-2.2241,2023,9
34,midcpnifty,9200,P,2023-09-18,2023-09-18 13:30:00,BUY,2023-09-18 13:35:00,50.4000,45.6000,52.3583,2023-09-18 13:55:00,52.3000,1.9000,1.5405,0.3595,3744.7917,TSL Hit,10416.6667,0.1498,2023,9
35,midcpnifty,9200,P,2023-09-18,2023-09-18 14:40:00,BUY,2023-09-18 14:45:00,49.8500,41.2500,50.7583,2023-09-18 15:05:00,47.4000,-2.4500,1.4587,-3.9088,-22725.2907,TSL Hit,5813.9535,-0.9090,2023,9
36,midcpnifty,9200,P,2023-09-18,2023-09-18 15:10:00,BUY,2023-09-18 15:15:00,52.2500,47.0000,47.0000,2023-09-18 15:20:00,50.6000,-1.6500,1.5427,-3.1927,-30407.1429,EOD Exit,9523.8095,-1.2163,2023,9
47,midcpnifty,9100,C,2023-09-25,2023-09-25 13:10:00,BUY,2023-09-25 13:15:00,32.6500,25.9000,25.9000,2023-09-25 13:15:00,25.9000,-6.7500,0.8782,-7.6282,-56505.5556,Initial SL Hit,7407.4074,-2.2602,2023,9
48,midcpnifty,9100,C,2023-09-25,2023-09-25 13:20:00,BUY,2023-09-25 13:25:00,31.7500,24.1000,27.0667,2023-09-25 13:45:00,24.1000,-7.6500,0.8377,-8.4877,-55475.4902,Initial SL hit,6535.9477,-2.2190,2023,9
49,midcpnifty,9100,C,2023-09-25,2023-09-25 14:05:00,BUY,2023-09-25 14:10:00,10.9000,6.4500,6.4500,2023-09-25 14:10:00,6.4500,-4.4500,0.2603,-4.7103,-52924.1573,Initial SL Hit,11235.9551,-2.1170,2023,9
50,midcpnifty,9100,C,2023-09-25,2023-09-25 14:15:00,BUY,2023-09-25 14:20:00,11.6000,5.5000,86.5750,2023-09-25 15:20:00,89.2500,77.6500,1.5127,76.1373,624075.8197,EOD Exit,8196.7213,24.9630,2023,9
67,midcpnifty,9000,C,2023-10-09,2023-10-09 14:30:00,BUY,2023-10-09 14:35:00,33.4000,21.4000,37.4000,2023-10-09 15:00:00,31.2500,-2.1500,0.9698,-3.1197,-12998.9583,TSL Hit,4166.6667,-0.5200,2023,10
78,midcpnifty,9200,P,2023-10-16,2023-10-16 14:05:00,BUY,2023-10-16 14:10:00,43.7000,31.4000,38.3000,2023-10-16 14:15:00,37.8000,-5.9000,1.2225,-7.1225,-28953.2520,TSL Hit,4065.0407,-1.1581,2023,10


In [178]:
import pandas as pd

# Initialize the DataFrame to store statistics
stats_df8 = pd.DataFrame(
    index=pd.date_range(start="2023-09", end="2024-06", freq="ME").strftime("%Y-%m"),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_expiry
# combined_df_sorted = tb_expiry_ce
# combined_df_sorted = tb_expiry_pe

for month in stats_df8.index:
    
    year, month_num = map(int, month.split('-'))
    month_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year) & (combined_df_sorted["Trade Month"] == month_num)]

    total_roi = month_trades["ROI%"].sum()

    total_trades = len(month_trades)

    win_rate = (month_trades["ROI%"] > 0).mean() * 100

    avg_profit = month_trades[month_trades["ROI%"] > 0]["ROI%"].mean()

    avg_loss = month_trades[month_trades["ROI%"] < 0]["ROI%"].mean()

    max_drawdown = (
        month_trades["ROI%"].cumsum() - month_trades["ROI%"].cumsum().cummax()
    ).min()

    roi_dd_ratio = total_roi / abs(max_drawdown) if max_drawdown != 0 else float('inf')

    stats_df8.loc[month] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2023-09,14.1672,8,25.0000,12.5564,-1.8243,-8.7215,1.6244
2023-10,14.8647,9,22.2222,11.6304,-1.1994,-3.1759,4.6805
2023-11,10.9937,6,33.3333,9.4444,-1.9738,-4.1331,2.6599
2023-12,-2.9605,3,33.3333,0.3193,-1.6399,-3.2799,-0.9026
2024-01,-8.0397,16,43.7500,1.5996,-2.1374,-13.0457,-0.6163
2024-02,12.0817,7,28.5714,9.7329,-1.4768,-5.0962,2.3707
2024-03,-0.3641,1,0.0000,NaN,-0.3641,0.0000,inf
2024-04,-3.0812,10,30.0000,3.6215,-1.9922,-10.2085,-0.3018
2024-05,73.6802,4,75.0000,25.3299,-2.3096,-2.3096,31.9014
Overall,111.3419,64.0000,34.3750,8.4136,-1.7561,-19.5342,5.6998


# Monthly Distribution of Expiry trades - Option Buying

In [772]:
# # tb_expiry.to_csv('Expiry Trading OS MTrend.csv')
# tb_targets = tb_expiry[tb_expiry['Remarks'] == 'Target Hit']
# len(tb_targets)